In [1]:
# imports
import pandas as pd
import numpy as np
import glob, os
import matplotlib.pyplot as plt

In [2]:
# fetch excel file names
path = "peg_standard"
excel_files = []
for file in os.listdir(path):
    if file.endswith(".xlsx"):
        excel_files.append(os.path.join(path, file))
print(excel_files)

['peg_standard/AH22-A1.xlsx', 'peg_standard/AH22-A2.xlsx', 'peg_standard/AH22-A3.xlsx', 'peg_standard/AH22-B1.xlsx', 'peg_standard/AH22-B2.xlsx', 'peg_standard/AH22-B3.xlsx', 'peg_standard/AH22-B4.xlsx', 'peg_standard/AH22-C1.xlsx', 'peg_standard/AH22-C2.xlsx', 'peg_standard/AH22-C3.xlsx', 'peg_standard/AH22-C4.xlsx']


In [3]:
# read in excel files into dataframes
dfs = []
for excel_file in excel_files:
    print(excel_file)
    xls = pd.ExcelFile(excel_file)
    dfs.append(pd.read_excel(xls, 'Raw Data', header=23)[23:].reset_index().drop('index', axis=1))

peg_standard/AH22-A1.xlsx
peg_standard/AH22-A2.xlsx
peg_standard/AH22-A3.xlsx
peg_standard/AH22-B1.xlsx
peg_standard/AH22-B2.xlsx
peg_standard/AH22-B3.xlsx
peg_standard/AH22-B4.xlsx
peg_standard/AH22-C1.xlsx
peg_standard/AH22-C2.xlsx
peg_standard/AH22-C3.xlsx
peg_standard/AH22-C4.xlsx


In [4]:
# truncate start data
truncate_index = []
for df in dfs:
    for i in range(len(df)):
        if (df.iloc[i]['Response Trace 4'] > -100):
            truncate_index.append(i)
            break
for i in range(len(dfs)):
    dfs[i] = dfs[i][truncate_index[i]:].dropna().reset_index().drop('index', axis=1)

In [9]:
dfs_new = []
for j in range(len(dfs)):
    rt_corrected = []
    for i in range(len(dfs[j])):
        x = dfs[j]['RT (mins)'][i]
        y_curr = dfs[j]['Response Trace 4'][i]
        rt_corrected.append(y_curr - (dfs[j].loc[0]['Response Trace 4'] + dfs[j].loc[len(dfs[j]) - 1]['Response Trace 4'])/(35)*x)
    dfs_new.append(pd.concat([dfs[j], pd.DataFrame({'RT Corrected': rt_corrected})], axis=1, sort=False))

,RT (mins),Response Trace 1,Response Trace 2,Response Trace 3,Response Trace 4,RT Corrected
0,0.383333,0.034021,0.037152,0.031146,28.610491,29.256348
1,0.400000,0.033282,0.031896,0.028695,28.594271,29.268209
2,0.416667,0.034453,0.029971,0.029218,28.757940,29.459958
3,0.433333,0.035234,0.029505,0.031969,29.256781,29.986880
4,0.450000,0.032558,0.029820,0.035113,30.119724,30.877904


In [10]:
# plot data and save as png 
for i in range(len(dfs)):
    plt.figure()
    plt.plot(dfs[i]['RT (mins)'], dfs[i]['Response Trace 1'], label='RT 1')
    #plt.plot(dfs[i]['RT (mins)'], dfs[i]['Response Trace 2'], label='RT 2')
    #plt.plot(dfs[i]['RT (mins)'], dfs[i]['Response Trace 3'], label='RT 3')
    plt.plot(dfs[i]['RT (mins)'], dfs[i]['Response Trace 4'], 'k-', label='RT 4')
    plt.plot(dfs_new[i]['RT (mins)'], dfs_new[i]['RT Corrected'], label='RT Corrected')
    plt.xlabel("time (mins)")
    plt.ylabel("nRIU")
    plt.legend()
    #plt.grid()
    plt.savefig(excel_files[i].replace(".xlsx", ""), dpi=300, bbox_inches='tight')